In [3]:
import pandas as pd 
# import  matplotlib.pyplot as plt
import numpy as np 
import tensorflow as tf
# import keras
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score , accuracy_score , confusion_matrix , f1_score
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# !pip install scikit-multilearn

In [4]:
train_data = pd.read_csv('./train.csv')

In [5]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
test_data = pd.read_csv('./test.csv')

In [7]:
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [9]:
test_target = pd.read_csv('./test_labels.csv')

In [10]:
test_target.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [11]:
comments = train_data.drop(['id', 'comment_text'], axis = 1)

In [12]:
comments

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [13]:
comments.columns[0]

'toxic'

In [14]:
for col in comments.columns:
    print('Percent of {0}s : '.format(col), round(100*comments[col].mean(), 2), ' %')

Percent of toxics :  9.58  %
Percent of severe_toxics :  1.0  %
Percent of obscenes :  5.29  %
Percent of threats :  0.3  %
Percent of insults :  4.94  %
Percent of identity_hates :  0.88  %


In [15]:
classes = {}

for col in comments.columns:
    classes[col] = comments[col].sum()

In [16]:
classes

{'toxic': 15294,
 'severe_toxic': 1595,
 'obscene': 8449,
 'threat': 478,
 'insult': 7877,
 'identity_hate': 1405}

In [17]:
# plt.bar(range(len(classes)), list(classes.values()), tick_label = list(classes.keys()))

In [18]:
def  clean_text(text):
    text =  text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation)) 
    text = re.sub("(\\W)"," ",text) 
    text = re.sub('\S*\d\S*\s*','', text)
    
    return text

In [19]:
train_data.comment_text = train_data.comment_text.apply(clean_text)

In [20]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww he matches this background colour i am se...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cannot make any real suggestions on im...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [21]:
nltk.download('stopwords')
sn = SnowballStemmer(language='english')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>


In [22]:
def stem_stopwords_reduction(text):
    text = [sn.stem(w) for w in text.split() if not w in set(stopwords.words('english'))]
    return ' '.join(text)

In [23]:
train_data.comment_text = train_data.comment_text.apply(stem_stopwords_reduction)

In [ ]:
# with open('stemmed_comment_text.pickle', 'wb') as file:
#     pickle.dump(train_data, file)

In [24]:
x = train_data.comment_text
y = train_data.drop(['id', 'comment_text'], axis = 1)

In [25]:
# x, y

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=45)

In [27]:
# x_train

In [28]:
train_data.shape

(159571, 8)

In [29]:
# y_train

In [30]:
word_vectorizer = TfidfVectorizer(
strip_accents='unicode', 
analyzer='word', 
stop_words='english', 
sublinear_tf=True, 
ngram_range=(1,3), 
token_pattern='\w{1,}'
)

word_vectorizer.fit(x_train)
train_word_features = word_vectorizer.transform(x_train)

In [31]:
# print(train_word_features[0])

In [32]:
X_train_transformed = word_vectorizer.transform(x_train)
X_test_transformed = word_vectorizer.transform(x_test)

In [33]:
# print(X_train_transformed)

In [35]:
import mlflow 
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"Trackign URI : '{mlflow.get_tracking_uri()}'")
print(mlflow.list_experiments())



Trackign URI : 'http://127.0.0.1:5000'
[<Experiment: artifact_location='./artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]


In [44]:
mlflow.set_experiment("toxic-comment-experiment-1")
with mlflow.start_run():
    mlflow.sklearn.autolog()
    params = {
        "C":10, 
        "penalty":"l2", 
        "solver":"liblinear",
        "random_state":45
    }
    log_reg = LogisticRegression(**params)

    classifier = OneVsRestClassifier(log_reg)
    classifier.fit(X_train_transformed, y_train)
    with open('./models_trial/classifier_model.pickle', 'wb') as file:
        pickle.dump(classifier, file)
        file.close()
    mlflow.log_artifacts("./models_trial", artifact_path="artifacts")

    y_train_pred_proba = classifier.predict_proba(X_train_transformed)
    y_test_pred_proba = classifier.predict_proba(X_test_transformed)
    
    # mlflow.log_metric('y_test_pred_proba', y_test_pred_proba)
    mlflow.sklearn.log_model(log_reg, artifact_path='artifacts')

    roc_auc_score_train = roc_auc_score(y_train, y_train_pred_proba,average='weighted')
    roc_auc_score_test = roc_auc_score(y_test, y_test_pred_proba,average='weighted')

    print("ROC AUC Score Train:", roc_auc_score_train)
    print("ROC AUC Score Test:", roc_auc_score_test)
    mlflow.log_metric("roc_auc_score_train", roc_auc_score_train)
    mlflow.log_metric("roc_auc_score_test", roc_auc_score_test)

2022/07/03 13:00:08 WARNING mlflow.sklearn.utils: Failed to import matplotlib (error: ModuleNotFoundError("No module named 'matplotlib'")). Skipping artifact logging.


ROC AUC Score Train: 0.9998057854773116
ROC AUC Score Test: 0.9776951654141759


In [ ]:
file.close()

In [ ]:
with open('classifier_model.pickle', 'wb') as file:
    pickle.dump(classifier, file)
    file.close()
    

In [ ]:
def make_test_predictions(text,classifier):
    
    df ={'id':[565],'comment_text':[text]}
    df = pd.DataFrame(df)
    df.comment_text = df.comment_text.apply(clean_text)
    df.comment_text = df.comment_text.apply(stem_stopwords_reduction)
    X_test = df.comment_text
    X_test_transformed = word_vectorizer.transform(X_test)
    y_test_pred = classifier.predict_proba(X_test_transformed)
    
    score_dict = {0:'Toxic', 1:'Severely Toxic', 2:'Obscene', 3:'Threat', 4:'Insult', 5:'Identity Hate'}
    offensive_score = np.amax(y_test_pred)
   
    if(offensive_score>0.7):
        print('Your comment is classified as {}, we suggest you to use a relatively normal tone'.format(score_dict[np.argmax(y_test_pred)]))
        
    elif(offensive_score>0.5):
        print("Your comment might be taken as offensive to some of our users, kindly change it")
        
    else:
        print('The comment/statement has no issues ')
#     return y_test_pred

In [ ]:
make_test_predictions("A walk in the park is the most refreshing thing I do in the morning", classifier)

The comment/statement has no issues 


In [ ]:
make_test_predictions("f@#k u a$$h0le", classifier)

Your comment is classified as Toxic, we suggest you to use a relatively normal tone


In [ ]:
make_test_predictions('shut up imbecile', classifier)

Your comment is classified as Toxic, we suggest you to use a relatively normal tone


In [ ]:
make_test_predictions('I think we should breakup', classifier)

The comment/statement has no issues 


In [ ]:
import time 
start = time.time()
make_test_predictions('This is the final test case for testing this model', classifier)
print('Time taken for generating the output is ', round((time.time()-start)*1000, 2), 'ms')

The comment/statement has no issues 
Time taken for generating the output is  70.8 ms


In [ ]:
with open('word_vectorizer.pickle', 'wb') as file:
    pickle.dump(word_vectorizer, file)
    file.close()